#  Step 1  A.Load Data: Import the dataset

In [1]:
import pandas as pd
Finaldata = pd.read_csv("/data/ManufactureFinalDatastes.csv")

In [2]:
print(Finaldata)

        Unnamed: 0 EquipmentID         Timestamp  Temperature  Vibration  \
0                0   Equip_007  01-01-2023 00:00     0.682891   0.370087   
1                1   Equip_007  01-01-2023 00:00     0.682891   0.370087   
2                2   Equip_007  01-01-2023 00:00     0.682891   0.370087   
3                3   Equip_007  01-01-2023 00:00     0.682891   0.370087   
4                4   Equip_007  01-01-2023 00:00     0.682891   0.370087   
...            ...         ...               ...          ...        ...   
249858      249858   Equip_013  28-07-2023 00:00     0.253512   0.964459   
249859      249859   Equip_013  28-07-2023 00:00     0.253512   0.964459   
249860      249860   Equip_013  28-07-2023 00:00     0.253512   0.964459   
249861      249861   Equip_013  28-07-2023 00:00     0.253512   0.964459   
249862      249862   Equip_013  28-07-2023 00:00     0.253512   0.964459   

        Pressure  OperationalLoad     RecordID   MaintenanceDate  \
0       0.145886   

In [3]:
Finaldata['time_since_last_maintenance'] = Finaldata['time_since_last_maintenance'].str[:3]

In [4]:
Finaldata['time_since_last_maintenance']

0         499
1         499
2         499
3         499
4         499
         ... 
249858    291
249859    291
249860    291
249861    291
249862    291
Name: time_since_last_maintenance, Length: 249863, dtype: object

In [5]:
Finaldata['time_since_last_maintenance'] = Finaldata['time_since_last_maintenance'].astype(int)

In [6]:
from tqdm import tqdm
dtypes = pd.DataFrame(Finaldata.dtypes)
dtypes.reset_index(inplace=True)

In [7]:
for i_ in tqdm(range(len(dtypes))):
    col_name = dtypes["index"].iloc[i_]
    if Finaldata[col_name].isna().sum() != 0:
        if Finaldata[col_name].dtype != "object":
            Finaldata[col_name].fillna(value=Finaldata[col_name].mean(), inplace=True)
        else:
            Finaldata[col_name].fillna(value=Finaldata[col_name].mode()[0], inplace=True)

100%|██████████| 24/24 [00:00<00:00, 275.05it/s]


In [8]:
print(Finaldata)

        Unnamed: 0 EquipmentID         Timestamp  Temperature  Vibration  \
0                0   Equip_007  01-01-2023 00:00     0.682891   0.370087   
1                1   Equip_007  01-01-2023 00:00     0.682891   0.370087   
2                2   Equip_007  01-01-2023 00:00     0.682891   0.370087   
3                3   Equip_007  01-01-2023 00:00     0.682891   0.370087   
4                4   Equip_007  01-01-2023 00:00     0.682891   0.370087   
...            ...         ...               ...          ...        ...   
249858      249858   Equip_013  28-07-2023 00:00     0.253512   0.964459   
249859      249859   Equip_013  28-07-2023 00:00     0.253512   0.964459   
249860      249860   Equip_013  28-07-2023 00:00     0.253512   0.964459   
249861      249861   Equip_013  28-07-2023 00:00     0.253512   0.964459   
249862      249862   Equip_013  28-07-2023 00:00     0.253512   0.964459   

        Pressure  OperationalLoad     RecordID   MaintenanceDate  \
0       0.145886   

In [9]:
list(Finaldata.columns)

['Unnamed: 0',
 'EquipmentID',
 'Timestamp',
 'Temperature',
 'Vibration',
 'Pressure',
 'OperationalLoad',
 'RecordID',
 'MaintenanceDate',
 'MaintenanceType',
 'Description',
 'MainType_new',
 '0',
 '1',
 '2',
 '3',
 'Temperature_7d_avg',
 'Vibration_7d_avg',
 'Pressure_7d_avg',
 'time_since_last_maintenance',
 'sum',
 'OperationalEfficiency',
 'OperationalLoad_7d_avg',
 'FailureFlag']

# B.Feature Selection

In [10]:
features = Finaldata.drop(columns=['EquipmentID', 'Timestamp', 'FailureFlag'])  # Adjust based on your target variable
target = Finaldata['FailureFlag']  # Or another continuous variable for regression


In [11]:
Finaldata_x = Finaldata[['Temperature','Vibration','Pressure','OperationalLoad','Temperature_7d_avg',
 'Vibration_7d_avg','Pressure_7d_avg','time_since_last_maintenance','sum','OperationalEfficiency','OperationalLoad_7d_avg' ]]
Finaldata_y = Finaldata['FailureFlag']
# X= training columns and Y is target column

In [12]:
Finaldata_y.head()

0    1
1    0
2    0
3    1
4    1
Name: FailureFlag, dtype: int64

# C.Split Data: Divide the dataset into training and testing sets.

In [13]:
X = Finaldata_x
Y = Finaldata_y


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Step 2: Initialize the Decision Tree Model

In [15]:
from sklearn.tree import DecisionTreeRegressor
tree_model = DecisionTreeRegressor(random_state=42)  #Create an Instance: Initialize the Decision Tree model 

# Step 3: Train the Model

In [16]:
tree_model.fit(X_train, Y_train)

DecisionTreeRegressor(random_state=42)

In [17]:
y_pred = tree_model.predict(X_test)

In [29]:
#Calculate model matrix
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 0.2501284123243217
R-squared: -0.0005544202854106395


In [28]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5, 10]
}

In [70]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [None, 70, 100, 150],
    'min_samples_split': [40, 80, 150, 200],
    'min_samples_leaf': [50, 100, 120, 150]
}

In [69]:
#Calculate model matrix
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 0.249995085577754
R-squared: -2.1091566849706922e-05


In [71]:
grid_search = GridSearchCV(tree_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
             param_grid={'max_depth': [None, 70, 100, 150],
                         'min_samples_leaf': [50, 100, 120, 150],
                         'min_samples_split': [40, 80, 150, 200]},
             scoring='neg_mean_squared_error')

In [65]:
best_tree_model = grid_search.best_estimator_

In [66]:
best_model_prediction = best_tree_model.predict(X_test)

In [67]:
y_pred = best_tree_model.predict(X_test)
mse = mean_squared_error(Y_test, y_pred)
r2 = r2_score(Y_test, y_pred)

In [68]:
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 0.249995085577754
R-squared: -2.1091566849706922e-05
